<a href="https://colab.research.google.com/github/OrJaku/road-sign-recognition/blob/master/road_sign_rec_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
cd drive/My\ Drive/Colab Notebooks/Road_sign_rec

/content/drive/My Drive/Colab Notebooks/Road_sign_rec


In [0]:
import os, shutil

### Przygotowanie folderów

In [0]:
base_dir = "Data/"
train_dir = os.path.join(base_dir, 'train_data')
try:
  os.mkdir(train_dir)
except FileExistsError:
  pass


cross_train =  os.path.join(train_dir, 'cross')
stop_train =  os.path.join(train_dir, 'stop')
try:
  os.mkdir(cross_train)
  os.mkdir(stop_train)
except FileExistsError:
  pass

valid_dir = os.path.join(base_dir, 'valid_data')
try:
  os.mkdir(valid_dir)
except FileExistsError:
  pass

cross_val =  os.path.join(valid_dir, 'cross')
stop_val =  os.path.join(valid_dir, 'stop')
try:
  os.mkdir(cross_val)
  os.mkdir(stop_val)
except FileExistsError:
  pass

test_dir = os.path.join(base_dir, 'test_data')
try:
  os.mkdir(test_dir)
except FileExistsError:
  pass

cross_test =  os.path.join(test_dir, 'cross')
stop_test =  os.path.join(test_dir, 'stop')
try:
  os.mkdir(cross_test)
  os.mkdir(stop_test)
except FileExistsError:
  pass



#### Ilość poszczegónych zdjęć

In [5]:
cross_origin_path = os.path.abspath("Data/information--pedestrians-crossing--g1/")
stop_origin_path = os.path.abspath("Data/regulatory--stop--g1")
print('Przejscie dla piszych: ',len(os.listdir(cross_origin_path)))
print('Stop: ', len(os.listdir(stop_origin_path)))

Przejscie dla piszych:  2188
Stop:  1368


#### Kopiowanie danych - znak przejścia dla pieszych

In [8]:
# train cross
for image_name in os.listdir(cross_origin_path)[0:1500]:
  src = os.path.join(cross_origin_path, image_name)
  dst = os.path.join(cross_train, image_name)
  shutil.copyfile(src, dst)
print("train cross copied")

# validation cross
for image_name in os.listdir(cross_origin_path)[1501:2000]:
  src = os.path.join(cross_origin_path, image_name)
  dst = os.path.join(cross_val, image_name)
  shutil.copyfile(src, dst)
print("validation cross copied")

# test cross
for image_name in os.listdir(cross_origin_path)[2001:]:
  src = os.path.join(cross_origin_path, image_name)
  dst = os.path.join(cross_test, image_name)
  shutil.copyfile(src, dst)
print("test cross copied")


train cross copied
validation cross copied
test cross copied


#### Kopiowanie danych - znak stop

In [10]:
# train stop
i=0
for image_name in os.listdir(stop_origin_path)[0:900]:
  i =+ 1
  src = os.path.join(stop_origin_path, image_name)
  dst = os.path.join(stop_train, image_name)
  shutil.copyfile(src, dst)
  if i == 200 or i == 400 or i == 700:
    print(f"Copied {i} ")
print("train stop copied")

# validation stop
for image_name in os.listdir(stop_origin_path)[901:1200]:
  src = os.path.join(stop_origin_path, image_name)
  dst = os.path.join(stop_val, image_name)
  shutil.copyfile(src, dst)
  if i == 1050:
    print(f"Copied {i} ")
print("validation stop copied")

# test stop
for image_name in os.listdir(stop_origin_path)[1201:]:
  src = os.path.join(stop_origin_path, image_name)
  dst = os.path.join(stop_test, image_name)
  shutil.copyfile(src, dst)
  if i == 1300:
    print(f"Copied {i} ")
print("test stop copied")

train stop copied
validation stop copied
test stop copied


#### Przekształcanie plików .jpg na tensory

In [12]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='binary'

)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='binary'

)

Found 2400 images belonging to 2 classes.
Found 798 images belonging to 2 classes.


In [0]:
import pandas as pd
import numpy as np
from keras import layers, models, optimizers


##Model

In [0]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPool2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPool2D((2, 2)))
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPool2D((2, 2)))
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPool2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.RMSprop(lr=1e-4),
    metrics=['acc']
    )



In [0]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data=valid_generator,
    validation_steps=50
)